# REGENERATION INDEX CALCULATION
---
July 5th, 2023 @irfansmf

---

### Import Module

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import xarray as xr

from scipy.signal import find_peaks

### Function RegIndex

In [ ]:
# Counting peak of precipitation in one day
def count_peak(data, beda_antar_peak):
    pdata = {'ch':data}
    df = pd.DataFrame(pdata)
    
    for i in range (len(df)):
        if df['ch'][i] < 1:
            df['ch'][i] = np.nan
            
    df['group_no'] = (np.isnan(df.ch)).cumsum()
    df = df.dropna()
    df_group = df.groupby('group_no')
    df_group_keys = np.squeeze([*df_group.groups.keys()])

    if len(df_group.groups) != 1:
        aa = {}
        ia = 0
        i = 0
        while i < len(df_group.groups):
            c = df_group.get_group(df_group_keys[i])
            j = i+1
            f = 0
            while (f == 0) and (j < len(df_group.groups)):
                d = df_group.get_group(df_group_keys[j])
                if (d.index[0] - c.index[-1]) <= beda_antar_peak:
                    c = pd.concat([c,d])
                    j = j+1
                else :
                    f = 1
            aa[ia] = c
            ia += 1
            i = j

    elif len(df_group.groups) == 1:
        aa = {}
        aa[0] = df
       
    return aa

# Checking peak
def check_peak(data, diff_max):
    if len(data) >= 2:
        for i in range (len(data)-1):
            if (data[i+1].index[0] - data[i].index[-1]) > diff_max:  # --> Checking based on time diff between 2 peaks
                del data[i]
    return data

### Reg Index Calculation

In [ ]:
df = pd.read_excel('Data/Precipitation Station Data.xlsx')
df.head(5)

In [ ]:
sdate = '2019-02-23 00:00'
edate = '2019-02-24 00:00'

idx1 = df[df['Time']== sdate].index.values[0]
idx2 = df[df['Time']== edate].index.values[0]
    
data = df['CH'][idx1:idx2].values

In [ ]:
st_2 = []
st_1 = []
diff = [3,6,9,12,15,18]  # --> 3: 30 minutes, etc
for k in diff:
    print('Diff-'+str(k))
    dff = count_peak(data, k)
    cdff = check_peak(dff, 36)  # --> 36 is 6 hour diff between peak 1 and peak 2 (max)
    if len(cdff) > 1:
        st_2.append(len(cdff))
    elif len(cdff) == 1:
        st_1.append(len(cdff))
    print(len(cdff))

RegI need more than 1 station.

\begin{align}
RegI = \frac{\sum Station_{peaks\ge 2}}{\sum Station_{peaks=1}}
\end{align}